## Prerequisite concepts in an Armory context

### Dataset

A dataset is a collection of images (samples) in a sequence-like structure such as a
tuple, map, or numpy array. Each can have a target (label) assigned. Datasets can be
imported from a variety of sources such as [PyTorch][pytorch], [Hugging Face][huggingface], or [GitHub][github].

[pytorch]: https://pytorch.org
[huggingface]: https://huggingface.co
[github]: https://github.com


#### Basic

The following is a basic example of loading tuple data:

In [ ]:
from charmory.data import TupleDataset

raw_dataset = [
        ([1, 2, 3], 4),
        ([5, 6, 7], 8),
    ]

keyed_dataset = TupleDataset(raw_dataset, x_key="data", y_key="target")

keyed_dataset[0]

You'll see that we have turned the raw dataset into a map with keys "data" and
"target". These keys are arbitrary; the same ones just need to be provided in
the evaluation later and correspond to the images and labels respectively.

#### Hugging Face

The following is an example of how to load a dataset from [Hugging Face][huggingface]:

[huggingface]: https://huggingface.co

In [ ]:
import functools

import datasets # Hugging Face dataset library

from charmory.data import ArmoryDataLoader
from transformers import AutoImageProcessor # Hugging Face image processor class

dataset = datasets.load_dataset("mnist", split="test")
processor = AutoImageProcessor.from_pretrained(
        "farleyknight-org-username/vit-base-mnist"  # Hugging Face model card
    )

def transform(processor, sample):
    # Use the HF image processor and convert from BW To RGB
    sample["image"] = processor([img.convert("RGB") for img in sample["image"]])[
        "pixel_values"
    ]
    return sample

dataset.set_transform(functools.partial(transform, processor))
dataloader = ArmoryDataLoader(dataset, batch_size=16, num_workers=5)

The `load_dataset` functions imports the [MNIST][mnist] (handwritten digit)
dataset from Hugging Face. The `split` parameter specifies which subset of the
dataset to load, is usually either `train` or `test` or possibly `validation`,
depending on the dataset.

The `processor` is needed to ensure the dataset is pre-processed into a form
that can be inputted into the model. The path passed into this function is a
Hugging Face model card location, but just the preprocessor is pulled in.

`AutoImageProcessor.from_pretrained` expects a Hugging Face name for the model
card. We use the [vit-base-mnist][mnist_model] model which corresponds to the dataset.

The function `transform` then cycles through the dataset and converts each image
into Hugging Face's 'RGB' form. The `set_transform` method is used the the
Hugging Face dataset and applied the transform function to the entire Hugging
Face dataset.

Then the PyTorch `ArmoryDataLoader` generates the numpy arrays that are
required by ART for the evaluation. `batch_size` and `num_workers` can be set per the system and user's needs and requirements.

[mnist]: https://huggingface.co/datasets/mnist
[mnist_model]: https://huggingface.co/farleyknight-org-username/vit-base-mnist

### Model

A model is the output of a machine learning algorithm run on a training set of
data. It is used to identify patterns or make predictions on unseen datasets.
Models can be imported from a variety of sources, including [Hugging Face][huggingface],
[GitHub][github], [PyPI][pypi], and jatic_toolbox.

[pypi]: https://pypi.org
[huggingface]: https://huggingface.co
[github]: https://github.com


#### Hugging Face

The following is an example of how to import a model from [Hugging Face][huggingface]:

In [ ]:
from transformers import AutoModelForImageClassification

from charmory.model.image_classification import JaticImageClassificationModel
from charmory.track import track_params

model = JaticImageClassificationModel(
        track_params(AutoModelForImageClassification.from_pretrained)(
            "farleyknight-org-username/vit-base-mnist"
        ),
    )

Here, `farleyknight-org-username/vit-base-mnist` is the Hugging Face model card
name. This model was trained on the same mnist dataset as we used above.
`track_params` is a function wrapper that stores the argument values as
parameters in MLflow and `JaticImageClassificationModel` is a wrapper to make
the model compatible with Armory.

In [ ]:
from jatic_toolbox import load_model 

model = track_params(load_model)(
        provider="torchvision",
        model_name="resnet34",
        task="image-classification",
    )

We then use the [`PyTorchClassifier`][pytorchclassifier] wrapper to make the model compatible with
the ART library. Note that this is specific to image classification models
written within the PyTorch framework. The parameters can be adjusted as needed. Here, we have `loss` being set to [`CrossEntropyLoss`][cel] from PyTorch's `nn` (neural network) class. This criterion computers the cross entropy loss between input logits and the target. The `optimizer` chosen is [`Adam`][adam] from PyTorch's `optim` (optimizer algorithms) class. It has its own set of parameters, to include `lr` which is the learning rate. The `input_shape` is simply the size of the input. `channels_first` specifies whether channels should be set first or last. In this case, True means they'll be set first. `nb_classes` is the number of prediction classes - in our case, 10. And finally, `clip_values` is the tuple form (min, max) of floats or np.ndarray representing the min/max values allowed for features. Since we're using single float values, these will be the range of all features.

`track_initial_params` is used so that these parameters are also tracked in
MLflow.

[pytorchclassifier]: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/estimators/classification.html#pytorch-classifier
[cel]: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
[adam]: https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam

In [ ]:
from art.estimators.classification import PyTorchClassifier

from charmory.track import track_init_params
import torch

classifier = track_init_params(PyTorchClassifier)(
    model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(3, 224, 224),
    channels_first=True,
    nb_classes=10,
    clip_values=(-1, 1),
)

#### Other

To see examples of importing models from other places such as GitHub and PyPI,
please see the auxiliary notebook, [Diving Deeper][colab-diving-deeper].

[colab-diving-deeper]: https://colab.research.google.com/github/twosixlabs/armory-library/blob/master/docs/diving_deeper.ipynb

### Attack

An attack is a transformation of (each sample in) the dataset in order to
disrupt the machine learning algorithm's results. For example, after an attack,
the model may misclassify an image or fail to detect an object. In Armory,
targeted attacks are designed to focus on only one class at a time. This is how
we test the adversarial robustness of a machine learning model.

Attacks can be loaded from [IBM's Adversarial Robustness Toolbox][art].

The following is an example of how to define an attack from ART's
ProjectedGradientDescent class. The Projected Gradient Descent attack is an
iterative method in which, after each iteration, the perturbation is projected
on an lp-ball of specified radius (in addition to clipping the values of the
adversarial sample so that it lies in the permitted data range). This is the
[attack proposed by Madry et al.][paper] for adversarial training.

The parameters here are as follows. The `classifier` is the type of model which is being attacked. `batch_size` is 1 which means only one adversarial example will be generated at a time. `eps` sets the epsilon value to .3 which is the size of the perturbation applied to the model. The goal of an attack is to try to change the model as much as possible with the smallest perturbation. `eps_step` is the attack step size, or input variation, at each iteration of the process. Here, we're using .007. `max_iter` is the maximum number of iterations of the process - in this case, 20. `num_random_init` is the number of random initializations within the epsilon ball. `random_eps` here is set to False which means we have a set epsilon. If this parameter were True, epsilon would be randomly drawn from a truncated normal distribution. (Literature suggests this for FGSM-based training to generalize across different epsilons, and then `eps_step` is modified to preserve the ratio of `eps`/`eps_step`)  Attacks can be targeted or untargeted which specifies whether the attack is designed to lead to a specific incorrect result or just anything other than the correct one. Here, we have `targeted` set to False which means we're just aiming for any wrong answer as output from the model. Finally, we have `verbose` set to False which means we don't need to see the progress bar and extra output.

[art]: https://github.com/Trusted-AI/adversarial-robustness-toolbox
[paper]: https://arxiv.org/abs/1706.06083

In [ ]:
from art.attacks.evasion import ProjectedGradientDescent

attack=track_init_params(ProjectedGradientDescent)(
    classifier,
    batch_size=1,
    eps=0.3,
    eps_step=0.007,
    max_iter=20,
    num_random_init=1,
    random_eps=False,
    targeted=False,
    verbose=False,
)

Again, `track_init_params` is used to output the initial metrics to MLflow. It
takes as input the default specs for this attack.

### Evaluation

Evaluations are what Armory is all about. They are essentially the testing of models with or without the application of an attack. An Armory evaluation contains all of the pertinent configuration details including the attack, dataset, model, metrics to collect, and host system configuration.

In [ ]:
import charmory.evaluation as ev
from armory.metrics.compute import BasicProfiler

evaluation = ev.Evaluation(
    name="mnist-vit-pgd",
    description="MNIST image classification using a ViT model and PGD attack",
    author="TwoSix",
    dataset=ev.Dataset(
        name="MNIST",
        x_key="data",
        y_key="target",
        test_dataloader=dataloader,
    ),
    model=ev.Model(
        name="ViT",
        model=classifier,
    ),
    attack=ev.Attack(
        name="PGD",
        attack=attack,
        use_label_for_untargeted=False,
    ),
    metric=ev.Metric(profiler=BasicProfiler()),
)

Here, we provide a name, description, and author of the evaluation. Then we create a Dataset for the evaluation using the `dataloader` defined previously around the Hugging Face data. The `x_key` and `y_key` names need to match what the dataset has. Similarly, the Model and Attack are based on the `classifier` model and `attack` defined previously. For the Metric, we'll use the `BasicProfiler` which outputs the average CPU time for each type of computation.

### Task

One of the last pieces here is the definition of the task to perform. Currently, Armory has two types of tasks: Image Classification and Object Detection. A task contains the evaluation configuration itself, as well as other details to include whether to skip the benign and/or attack datasets, an optional adapter to be applied to the inference data prior to exporting to MLflow, and a frequency at which batches will be exported to MLflow, if at all. The adapter we're using in this case is `Unnormalize` which is the inverse of the `torchvision.transforms.Normalize` transform. It takes as parameters a mean and standard deviation.

In [ ]:
from charmory.tasks.image_classification import ImageClassificationTask
from charmory.utils import Unnormalize

task = ImageClassificationTask(
        evaluation,
        num_classes=10,
        export_adapter=Unnormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
        export_every_n_batches=10,
    )

### Engine

Finally, to actually run an evaluation, an Engine needs to be created. In Armory, there are currently two choices of engines: `EvaluationEngine` for performing model robustness evaluations as it pertains to adversarial attacks, and `AdversarialDatasetEngine` which creates an adversarial dataset by applying an attack to each sample in the original dataset. In this case, we already have an adversarial dataset, so we just need to start an `EvaluationEngine`. It takes as parameters the `task` which we defined above and `limit_test_batches` which we set to 16. 

In [ ]:
from charmory.engine import EvaluationEngine

engine = EvaluationEngine(task, limit_test_batches=16)